In [2]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 6.6MB/s 


In [4]:
import os
import json
import string

import numpy as np

from nltk.tag import pos_tag

from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import make_scorer,confusion_matrix
from sklearn.metrics import f1_score,classification_report
from sklearn.pipeline import Pipeline

from pprint import pprint

from keras.preprocessing.text import Tokenizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
train_loc = "/content/drive/My Drive/Data/NLP/Practical_NLP_Oreilly/Chap6/snips/train_PlayMusic_full.json"
test_loc = "/content/drive/My Drive/Data/NLP/Practical_NLP_Oreilly/Chap6/snips/validate_PlayMusic.json"

train_file = json.load(open(train_loc, encoding= "iso-8859-2"))
test_file = json.load(open(test_loc, encoding= "iso-8859-2"))

In [11]:
train_datafile = [i["data"] for i in train_file["PlayMusic"]]
test_datafile = [i["data"] for i in test_file["PlayMusic"]]

In [16]:
print(len(train_datafile))
print(len(test_datafile))

2000
100


In [13]:
train_datafile[:3]

[[{'text': 'I need to hear the '},
  {'entity': 'music_item', 'text': 'song'},
  {'text': ' '},
  {'entity': 'track', 'text': 'Aspro Mavro'},
  {'text': ' from '},
  {'entity': 'artist', 'text': 'Bill Szymczyk'},
  {'text': ' on '},
  {'entity': 'service', 'text': 'Youtube'}],
 [{'text': 'play '},
  {'entity': 'track', 'text': 'Yo Ho'},
  {'text': ' from '},
  {'entity': 'artist', 'text': 'the new york pops'},
  {'text': ' on '},
  {'entity': 'service', 'text': 'Youtube'}],
 [{'text': 'Play some '},
  {'entity': 'year', 'text': 'seventies'},
  {'text': ' music by '},
  {'entity': 'artist', 'text': 'Janne Puurtinen'},
  {'text': ' on '},
  {'entity': 'service', 'text': 'Youtube'},
  {'text': '.'}]]

In [14]:
test_datafile[:3]

[[{'text': 'can you put on '},
  {'entity': 'album', 'text': 'Like A Hurricane'},
  {'text': ' by '},
  {'entity': 'artist', 'text': 'Paul Landers'}],
 [{'text': 'Play '},
  {'entity': 'album', 'text': 'The Happy Blues'},
  {'text': ' by '},
  {'entity': 'artist', 'text': 'Ronnie Wood'},
  {'text': '.'}],
 [{'text': 'Play the '},
  {'entity': 'sort', 'text': 'newest'},
  {'text': ' '},
  {'entity': 'music_item', 'text': 'melody'},
  {'text': ' on '},
  {'entity': 'service', 'text': 'Last Fm'},
  {'text': ' by '},
  {'entity': 'artist', 'text': 'Eddie Vinson'},
  {'text': '.'}]]

In [17]:
def convert_data(datalist):
    output = []
    for data in datalist:
        sent = []
        pos = []
        for phrase in data:
            words = phrase["text"].strip().split(" ")
            while "" in words:
                words.remove("")
            if "entity" in phrase.keys():
                label = phrase["entity"]
                labels = [label+"-{}".format(i+1) for i in range(len(words))]
            else:
                labels = ["O"] * len(words)
            sent.extend(words)
            pos.extend(labels)
        output.append([sent, pos])
        print(sent)
    return output

In [18]:
train_data = convert_data(train_datafile)

['I', 'need', 'to', 'hear', 'the', 'song', 'Aspro', 'Mavro', 'from', 'Bill', 'Szymczyk', 'on', 'Youtube']
['play', 'Yo', 'Ho', 'from', 'the', 'new', 'york', 'pops', 'on', 'Youtube']
['Play', 'some', 'seventies', 'music', 'by', 'Janne', 'Puurtinen', 'on', 'Youtube', '.']
['play', 'the', 'MĂşsica', 'Da', 'SĂŠrie', 'De', 'Filmes', 'O', 'Hobbit', 'album', 'by', 'Alex', 'Otaola']
['Play', 'Magic', 'Sam', 'from', 'the', 'thirties']
['Play', 'The', 'Soft', 'Parade', '.']
['Play', 'music', 'by', 'blowfly', 'from', 'the', 'seventies']
['play', 'Jeff', 'Pilson', 'on', 'Youtube']
['play', 'a', 'tune', 'by', 'Syreeta', 'Wright', 'from', 'twenties', 'from', 'the', 'top']
['Please', 'play', 'me', 'Jerry', 'Lee', 'Lewis', "'s", 'If', 'You', 'Say', 'So', 'track', '.']
['play', 'the', 'newest', 'by', 'Exuma']
['Play', 'the', 'album', 'alas', 'y', 'raĂ\xadces', 'by', 'Dave', 'Pybus', '.']
['play', 'music', 'by', 'Helen', 'Ward']
['Play', 'some', 'sixties', 'music.']
['Plan', 'an', 'album', 'by', 'Roni',

In [50]:
train_data[0]

[['I',
  'need',
  'to',
  'hear',
  'the',
  'song',
  'Aspro',
  'Mavro',
  'from',
  'Bill',
  'Szymczyk',
  'on',
  'Youtube'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'music_item-1',
  'track-1',
  'track-2',
  'O',
  'artist-1',
  'artist-2',
  'O',
  'service-1']]

In [19]:
test_data = convert_data(test_datafile)

['can', 'you', 'put', 'on', 'Like', 'A', 'Hurricane', 'by', 'Paul', 'Landers']
['Play', 'The', 'Happy', 'Blues', 'by', 'Ronnie', 'Wood', '.']
['Play', 'the', 'newest', 'melody', 'on', 'Last', 'Fm', 'by', 'Eddie', 'Vinson', '.']
['Use', 'Groove', 'Shark', 'to', 'play', 'music']
['Please', 'play', 'something', 'good', 'from', 'U-roy', '.', 'Any', 'song', 'from', '1975', 'on', 'Zvooq', 'will', 'do.']
['Play', 'a', 'symphony', 'from', '2013', '.']
['Let', 'me', 'hear', 'the', 'good', 'songs', 'from', 'James', 'Iha']
['Play', 'my', 'inventive', 'playlist']
['I', 'want', 'to', 'play', 'music', 'from', 'iheart']
['Play', 'Subconscious', 'Lobotomy', 'from', 'Jennifer', 'Paull', '.']
['I', 'want', 'to', 'hear', 'a', 'seventies', 'sound', 'track', '.']
['play', 'a', 'john', 'maher', 'track']
['Please', 'play', 'something', 'from', 'Dihan', 'Slabbert', "that's", 'on', 'the', 'top', 'fifty']
['Please', 'play', 'something', 'catchy', 'on', 'Youtube']
['Play', 'something', 'from', '2004', 'by', 'Imo

In [51]:
test_data[0]

[['can',
  'you',
  'put',
  'on',
  'Like',
  'A',
  'Hurricane',
  'by',
  'Paul',
  'Landers'],
 ['O',
  'O',
  'O',
  'O',
  'album-1',
  'album-2',
  'album-3',
  'O',
  'artist-1',
  'artist-2']]

In [20]:
#BASE_DIR = 'Data'
#GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
GLOVE_DIR='/content/drive/My Drive/Data/NLP/glove.6B.100d.txt'
MAX_SEQUENCE_LENGTH = 300
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.3

In [22]:
print('Preparing embedding matrix.')

# first, build index mapping words in the embeddings set
# to their embedding vector

embeddings_index = {}
with open(os.path.join(GLOVE_DIR), encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))

def get_embeddings(word):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is None:
        # words not found in embedding index will be all-zeros.
        embedding_vector = np.zeros(shape=(EMBEDDING_DIM, ))
    return embedding_vector

Preparing embedding matrix.
Found 400000 word vectors in Glove embeddings.


In [44]:
train_texts = [" ".join(i[0]) for i in train_data]
test_texts = [" ".join(i[0]) for i in test_data]

train_texts[:3]

['I need to hear the song Aspro Mavro from Bill Szymczyk on Youtube',
 'play Yo Ho from the new york pops on Youtube',
 'Play some seventies music by Janne Puurtinen on Youtube .']

In [24]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts) #Converting text to a vector of word indexes
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 2866 unique tokens.


In [26]:
train_sequences[:3]

[[11, 119, 9, 12, 2, 10, 743, 744, 3, 213, 745, 6, 35],
 [1, 746, 385, 3, 2, 57, 747, 748, 6, 35],
 [1, 8, 66, 5, 4, 749, 750, 6, 35]]

In [27]:
"""
Get features for all words in the sentence
Features:
- word context: a window of 2 words on either side of the current word, and current word.
- POS context: a window of 2 POS tags on either side of the current word, and current tag. 
input: sentence as a list of tokens.
output: list of dictionaries. each dict represents features for that word.
"""
def sent2feats(sentence):
    feats = []
    sen_tags = pos_tag(sentence) #This format is specific to this POS tagger!
    for i in range(0,len(sentence)):
        word = sentence[i]
        wordfeats = {}
       #word features: word, prev 2 words, next 2 words in the sentence.
        wordfeats['word'] = word
        if i == 0:
            wordfeats["prevWord"] = wordfeats["prevSecondWord"] = "<S>"
        elif i==1:
            wordfeats["prevWord"] = sentence[0]
            wordfeats["prevSecondWord"] = "</S>"
        else:
            wordfeats["prevWord"] = sentence[i-1]
            wordfeats["prevSecondWord"] = sentence[i-2]
        #next two words as features
        if i == len(sentence)-2:
            wordfeats["nextWord"] = sentence[i+1]
            wordfeats["nextNextWord"] = "</S>"
        elif i==len(sentence)-1:
            wordfeats["nextWord"] = "</S>"
            wordfeats["nextNextWord"] = "</S>"
        else:
            wordfeats["nextWord"] = sentence[i+1]
            wordfeats["nextNextWord"] = sentence[i+2]
        
        #POS tag features: current tag, previous and next 2 tags.
        wordfeats['tag'] = sen_tags[i][1]
        if i == 0:
            wordfeats["prevTag"] = wordfeats["prevSecondTag"] = "<S>"
        elif i == 1:
            wordfeats["prevTag"] = sen_tags[0][1]
            wordfeats["prevSecondTag"] = "</S>"
        else:
            wordfeats["prevTag"] = sen_tags[i - 1][1]

            wordfeats["prevSecondTag"] = sen_tags[i - 2][1]
            # next two words as features
        if i == len(sentence) - 2:
            wordfeats["nextTag"] = sen_tags[i + 1][1]
            wordfeats["nextNextTag"] = "</S>"
        elif i == len(sentence) - 1:
            wordfeats["nextTag"] = "</S>"
            wordfeats["nextNextTag"] = "</S>"
        else:
            wordfeats["nextTag"] = sen_tags[i + 1][1]
            wordfeats["nextNextTag"] = sen_tags[i + 2][1]
            
        #Adding word vectors
        vector = get_embeddings(word)
        for iv,value in enumerate(vector):
            wordfeats['v{}'.format(iv)]=value
        
        
        feats.append(wordfeats)
    return feats

In [28]:
#Extract features from the conll data, after loading it.
def get_feats_conll(conll_data):
    feats = []
    labels = []
    for sentence in conll_data:
        feats.append(sent2feats(sentence[0]))
        labels.append(sentence[1])
    return feats, labels

In [29]:
#Train a sequence model
def train_seq(X_train,Y_train,X_dev,Y_dev):
    crf = CRF(algorithm='lbfgs', c1=0.1, c2=10, max_iterations=50)#, all_possible_states=True)
    #Just to fit on training data
    crf.fit(X_train, Y_train)
    labels = list(crf.classes_)
    #testing:
    y_pred = crf.predict(X_dev)
    sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
    print(metrics.flat_f1_score(Y_dev, y_pred,average='weighted', labels=labels))
    print(metrics.flat_classification_report(Y_dev, y_pred, labels=sorted_labels, digits=3))
    #print(metrics.sequence_accuracy_score(Y_dev, y_pred))
    get_confusion_matrix(Y_dev, y_pred,labels=sorted_labels)


In [30]:
#source for this function: https://gist.github.com/zachguo/10296432
def print_cm(cm, labels):
    print("\n")
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        sum = 0
        for j in range(len(labels)):
            cell = "%{0}.0f".format(columnwidth) % cm[i, j]
            sum =  sum + int(cell)
            print(cell, end=" ")
        print(sum) #Prints the total number of instances per cat at the end.


In [31]:
#python-crfsuite does not have a confusion matrix function, 
#so writing it using sklearn's confusion matrix and print_cm from github
def get_confusion_matrix(y_true,y_pred,labels):
    trues,preds = [], []
    for yseq_true, yseq_pred in zip(y_true, y_pred):
        trues.extend(yseq_true)
        preds.extend(yseq_pred)
    print_cm(confusion_matrix(trues,preds,labels),labels)


In [33]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [49]:
train_data

[[['I',
   'need',
   'to',
   'hear',
   'the',
   'song',
   'Aspro',
   'Mavro',
   'from',
   'Bill',
   'Szymczyk',
   'on',
   'Youtube'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'music_item-1',
   'track-1',
   'track-2',
   'O',
   'artist-1',
   'artist-2',
   'O',
   'service-1']],
 [['play', 'Yo', 'Ho', 'from', 'the', 'new', 'york', 'pops', 'on', 'Youtube'],
  ['O',
   'track-1',
   'track-2',
   'O',
   'artist-1',
   'artist-2',
   'artist-3',
   'artist-4',
   'O',
   'service-1']],
 [['Play',
   'some',
   'seventies',
   'music',
   'by',
   'Janne',
   'Puurtinen',
   'on',
   'Youtube',
   '.'],
  ['O',
   'O',
   'year-1',
   'O',
   'O',
   'artist-1',
   'artist-2',
   'O',
   'service-1',
   'O']],
 [['play',
   'the',
   'MĂşsica',
   'Da',
   'SĂŠrie',
   'De',
   'Filmes',
   'O',
   'Hobbit',
   'album',
   'by',
   'Alex',
   'Otaola'],
  ['O',
   'O',
   'album-1',
   'album-2',
   'album-3',
   'album-4',
   'album-5',
   'album-6',
   'album-7',
   'mus

In [34]:
feats, labels = get_feats_conll(train_data)
devfeats, devlabels = get_feats_conll(test_data)

Training a Sequence classification model with CRF


In [45]:
train_texts[0]

'I need to hear the song Aspro Mavro from Bill Szymczyk on Youtube'

In [47]:
print(feats[0])
print(feats[5])

[{'word': 'I', 'prevWord': '<S>', 'prevSecondWord': '<S>', 'nextWord': 'need', 'nextNextWord': 'to', 'tag': 'PRP', 'prevTag': '<S>', 'prevSecondTag': '<S>', 'nextTag': 'VBP', 'nextNextTag': 'TO', 'v0': 0.0, 'v1': 0.0, 'v2': 0.0, 'v3': 0.0, 'v4': 0.0, 'v5': 0.0, 'v6': 0.0, 'v7': 0.0, 'v8': 0.0, 'v9': 0.0, 'v10': 0.0, 'v11': 0.0, 'v12': 0.0, 'v13': 0.0, 'v14': 0.0, 'v15': 0.0, 'v16': 0.0, 'v17': 0.0, 'v18': 0.0, 'v19': 0.0, 'v20': 0.0, 'v21': 0.0, 'v22': 0.0, 'v23': 0.0, 'v24': 0.0, 'v25': 0.0, 'v26': 0.0, 'v27': 0.0, 'v28': 0.0, 'v29': 0.0, 'v30': 0.0, 'v31': 0.0, 'v32': 0.0, 'v33': 0.0, 'v34': 0.0, 'v35': 0.0, 'v36': 0.0, 'v37': 0.0, 'v38': 0.0, 'v39': 0.0, 'v40': 0.0, 'v41': 0.0, 'v42': 0.0, 'v43': 0.0, 'v44': 0.0, 'v45': 0.0, 'v46': 0.0, 'v47': 0.0, 'v48': 0.0, 'v49': 0.0, 'v50': 0.0, 'v51': 0.0, 'v52': 0.0, 'v53': 0.0, 'v54': 0.0, 'v55': 0.0, 'v56': 0.0, 'v57': 0.0, 'v58': 0.0, 'v59': 0.0, 'v60': 0.0, 'v61': 0.0, 'v62': 0.0, 'v63': 0.0, 'v64': 0.0, 'v65': 0.0, 'v66': 0.0, 'v67': 0.0

In [56]:
print(len(feats[0]))
print(len(feats[0][0]))
print(len(feats[0][1]))

13
110
110


In [54]:
feats[0]

13


[{'nextNextTag': 'TO',
  'nextNextWord': 'to',
  'nextTag': 'VBP',
  'nextWord': 'need',
  'prevSecondTag': '<S>',
  'prevSecondWord': '<S>',
  'prevTag': '<S>',
  'prevWord': '<S>',
  'tag': 'PRP',
  'v0': 0.0,
  'v1': 0.0,
  'v10': 0.0,
  'v11': 0.0,
  'v12': 0.0,
  'v13': 0.0,
  'v14': 0.0,
  'v15': 0.0,
  'v16': 0.0,
  'v17': 0.0,
  'v18': 0.0,
  'v19': 0.0,
  'v2': 0.0,
  'v20': 0.0,
  'v21': 0.0,
  'v22': 0.0,
  'v23': 0.0,
  'v24': 0.0,
  'v25': 0.0,
  'v26': 0.0,
  'v27': 0.0,
  'v28': 0.0,
  'v29': 0.0,
  'v3': 0.0,
  'v30': 0.0,
  'v31': 0.0,
  'v32': 0.0,
  'v33': 0.0,
  'v34': 0.0,
  'v35': 0.0,
  'v36': 0.0,
  'v37': 0.0,
  'v38': 0.0,
  'v39': 0.0,
  'v4': 0.0,
  'v40': 0.0,
  'v41': 0.0,
  'v42': 0.0,
  'v43': 0.0,
  'v44': 0.0,
  'v45': 0.0,
  'v46': 0.0,
  'v47': 0.0,
  'v48': 0.0,
  'v49': 0.0,
  'v5': 0.0,
  'v50': 0.0,
  'v51': 0.0,
  'v52': 0.0,
  'v53': 0.0,
  'v54': 0.0,
  'v55': 0.0,
  'v56': 0.0,
  'v57': 0.0,
  'v58': 0.0,
  'v59': 0.0,
  'v6': 0.0,
  'v60': 0

In [53]:
labels[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'music_item-1',
 'track-1',
 'track-2',
 'O',
 'artist-1',
 'artist-2',
 'O',
 'service-1']

In [42]:
print(labels[0])
print(labels[5])

['O', 'O', 'O', 'O', 'O', 'music_item-1', 'track-1', 'track-2', 'O', 'artist-1', 'artist-2', 'O', 'service-1']
['O', 'track-1', 'track-2', 'track-3', 'O']


In [48]:
print("Training a Sequence classification model with CRF")
train_seq(feats, labels, devfeats, devlabels)
print("Done with sequence model")

Training a Sequence classification model with CRF
0.8560889758746073
              precision    recall  f1-score   support

           O      0.976     0.983     0.980       418
      year-1      0.960     0.960     0.960        25
     genre-1      1.000     0.333     0.500         3
     genre-2      1.000     0.333     0.500         3
     genre-3      0.000     0.000     0.000         0
     genre-4      0.000     0.000     0.000         0
     genre-5      0.000     0.000     0.000         0
     genre-6      0.000     0.000     0.000         0
   service-1      0.857     0.923     0.889        39
   service-2      1.000     0.818     0.900        11
  playlist-1      1.000     0.333     0.500         9
  playlist-2      0.333     0.167     0.222         6
  playlist-3      0.000     0.000     0.000         4
  playlist-4      0.000     0.000     0.000         4
  playlist-5      0.000     0.000     0.000         2
  playlist-6      0.000     0.000     0.000         0
     album-1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
